In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import anthropic
import shutil
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"
openai = OpenAI()
claude = anthropic.Anthropic()

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [2]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": message})
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

# gr.ChatInterface(fn=chat).launch()

In [3]:
ticket_prices = {"london": "$799",
                 "paris": "$899",
                 "tokyo": "$1400",
                 "berlin": "$499"}

def get_ticket_price(destination_city):
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

get_ticket_price("Berlin")

'$499'

In [4]:
booking_confirmations = {
    "london": "The booking to London is confirmed with the ID number 1.",
    "paris": "The booking to Paris is confirmed with the ID number 2.",
    "tokyo": "The booking to Tokyo is confirmed with the ID number 3.",
    "berlin": "The booking to Berlin is confirmed with the ID number 4.",
}

def confirm_booking(destination_city):
    print(f"Tool confirm_booking called for {destination_city}")
    city = destination_city.lower()
    return booking_confirmations.get(city, f"No available bookings for the chosen destination: {destination_city}")

confirm_booking("Berlin")

Tool confirm_booking called for Berlin


'The booking to Berlin is confirmed with the ID number 4.'

In [5]:
claude_system_message = "Given some text you translate it to Russian. Do not add any comments or additional information."

def translate_to_russian(text):
    print("CALLING TRANSLATION FUNCTION")
    result = claude.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0,
        system=claude_system_message,
        messages=[
            {"role": "user", "content": text}
        ]
    )
    return result.content[0].text

translate_to_russian("Hello world")

CALLING TRANSLATION FUNCTION


'Здравствуй, мир'

In [6]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city" : {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}


booking_confirmation_function = {
    "name": "confirm_booking",
    "description": "Confirm that a booking was made for the given destination city. Call this whenever you need to confirm booking",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city in which the customer wants a booking"
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}


translate_to_russian_function = {
    "name": "translate_to_russian",
    "description": "Call this function whenever you are asked to translate the given text to Russian.",
    "parameters": {
        "type": "object",
        "properties": {
            "text_to_translate": {
                "type": "string",
                "description": "The text to be translated to Russian"
            }
        },
        "required": ["text_to_translate"],
        "additionalProperties": False
    }
}


tools = [{"type": "function", "function": price_function},
         {"type": "function", "function": booking_confirmation_function},
         {"type": "function", "function": translate_to_russian_function}]

In [7]:
# def chat(message, history):
#     messages = [{"role": "system", "content": system_message}]
#     for human, assistant in history:
#         messages.append({"role": "user", "content": human})
#         messages.append({"role": "assistant", "content": assistant})
#     messages.append({"role": "user", "content": message})
#     response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

#     if response.choices[0].finish_reason == "tool_calls":
#         message = response.choices[0].message
#         # message.tool_calls = [message.tool_calls[0]]
#         response, city = handle_tool_call(message)
#         messages.append(message)
#         messages.append(response)
#         print(f"Messages after appending the message and response: {messages}")
#         response = openai.chat.completions.create(model=MODEL, messages=messages)

#     return response.choices[0].message.content


# def handle_tool_call(message):
#     tool_call = message.tool_calls[0]
#     arguments = json.loads(tool_call.function.arguments)
#     city = arguments.get("destination_city")

#     if tool_call.function.name == "get_ticket_price":
#         key = "price"
#         value = get_ticket_price(city)
#     else:
#         key = "confirmation"
#         value = confirm_booking(city)  
#     print(f"HERE: {message.tool_calls}")
#     response = {
#         "role": "tool",
#         "content": json.dumps({"destination_city": city, key: value}),
#         "tool_call_id": message.tool_calls[0].id
#     }

#     return response, city


# gr.ChatInterface(fn=chat).launch()








def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": message})
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        # message.tool_calls = [message.tool_calls[0]]
        responses, cities, translations = handle_tool_calls(message)
        messages.append(message)
        for tool_response in responses:
            messages.append(tool_response)
        # print(f"Messages after appending the message and response: {messages}")
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content


def handle_tool_calls(message):

    def handle_function_call(function_name, argument):
        actions = {
            "get_ticket_price": {"key": "price", "function": get_ticket_price},
            "confirm_booking": {"key": "confirmation", "function": confirm_booking},
            "translate_to_russian": {"key": "translation", "function": translate_to_russian}
        }

        if function_name not in actions:
            raise ValueError(f"Unknown function name: {function_name}")
        
        action = actions[function_name]
        return {action["key"]: action["function"](argument)}

    response_primary_key_dict = {key: "destination_city" for key in booking_confirmations.keys()}

    responses = []
    cities = []
    translations = []

    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        
        argument = next(iter(arguments.values())).lower()

        result = handle_function_call(tool_call.function.name, argument)

        primary_key = response_primary_key_dict.get(argument, "text_to_translate")
        response_key, response_value = next(iter(result.items()))
        
        response = {
            "role": "tool",
            "content": json.dumps({primary_key: argument, response_key: response_value}),
            "tool_call_id": tool_call.id
        }
        
        responses.append(response)
        if arguments.get("destination_city"): cities.append(arguments.get("destination_city"))
        if arguments.get("text_to_translate"): translations.append(response_value)
    
        

    return responses, cities, translations


gr.ChatInterface(fn=chat).launch()

c:\Users\veace\anaconda3\envs\llms\Lib\site-packages\gradio\components\chatbot.py:279: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [8]:
import base64
from io import BytesIO
from PIL import Image

def artist(cities):
    unique_cities = set(cities)
    combined_cities = ", ".join(unique_cities)
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=f"An collage representing a vacation in {combined_cities}, showing tourist spots and everything unique about {combined_cities}, in a vibrant pop-art style. The collage must be divided into {len(combined_cities)} distincy sections, one section for each of the cities. Each city in the collage has to ocupy ots own section without overlapping with other cities' sections",
        size = "1024x1024",
        n=1,
        response_format="b64_json"
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))


# image = artist(["Dublin", "Chisinau"])

# display(image)

In [9]:
from pydub import AudioSegment
from pydub.playback import play
import pygame

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message
    )
    audio_stream = BytesIO(response.content)
    pygame.init()
    pygame.mixer.init()
    sound = pygame.mixer.Sound(audio_stream)
    sound.play()
    while pygame.mixer.get_busy():
        pygame.time.delay(100)
    pygame.quit()

# talker("Well, hi there")

pygame 2.6.1 (SDL 2.28.4, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [10]:
# def chat(message, history):
#     image = None
#     conversation = [{"role": "system", "content": system_message}]
#     for human, assistant in history:
#         conversation.append({"role": "user", "content": human})
#         conversation.append({"role": "assistant", "content": assistant})
#     conversation.append({"role": "user", "content": message})
#     response = openai.chat.completions.create(model=MODEL, messages=conversation, tools=tools)

#     if response.choices[0].finish_reason=="tool_calls":
#         message = tool_call = response.choices[0].message
#         print(f"This is the message: {message}")
#         response, city = handle_tool_calls(message)
#         conversation.append(message)
#         conversation.append(response)
#         image = artist(city)
#         response = openai.chat.completions.create(model=MODEL, messages=conversation)

#     reply = response.choices[0].message.content
#     talker(reply)
#     return reply, image






def chat(message, history):
    image = None
    translations = None
    conversation = [{"role": "system", "content": system_message}]

    for human, assistant in history:
        conversation.append({"role": "user", "content": human})
        conversation.append({"role": "assistant", "content": assistant})
    conversation.append({"role": "user", "content": message})
    response = openai.chat.completions.create(model=MODEL, messages=conversation, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses, cities, translations = handle_tool_calls(message)
        print(f"TRANSLATIONS: {translations}")
        conversation.append(message)
        for tool_response in responses:
            conversation.append(tool_response)
        if cities:
            image = artist(cities)
        response = openai.chat.completions.create(model=MODEL, messages=conversation)

    reply = response.choices[0].message.content
    talker(reply)
    return reply, image, translations

In [14]:


with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500)
        image_output = gr.Image(height=500)
        translation_output = gr.Textbox(label="Translation to Russian:", interactive=False)
    with gr.Row():
        msg = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        audio_input = gr.Audio(type="filepath", label="Record or Upload Audio")
    with gr.Row():
        clear = gr.Button("Clear")

    audio_state = gr.State()

    def user(user_message, history):
        if user_message:
            return "", history + [[user_message, None]]
        else:
            return "", history + [[None, None]]
    

    def process_audio(audio_filepath="saved_audio/audio_file.wav"):
        if audio_filepath: 
            save_path = "saved_audio/audio_file.wav"
            shutil.copy(audio_filepath, save_path)
            audio_file = open("saved_audio/audio_file.wav", "rb")
            transcription = openai.audio.transcriptions.create(
                model="whisper-1", 
                file=audio_file
            )
            print(transcription.text)
            return transcription.text
        return None
        
    
    def bot(history):
        user_message = history[-1][0]
        if user_message:
            bot_message, image, translation = chat(user_message, history[:-1])
            translation_to_display = "\n".join(translation) if translation else gr.update()
            image_to_display = image if image else gr.update()
            history[-1][1] = bot_message if not translation else "See translation in the box on the right."
            return history, image_to_display, translation_to_display
        else:
            return history[:-1], gr.update(), gr.update()
    
    

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, [chatbot, image_output, translation_output]
    )


    audio_input.change(process_audio, inputs=[audio_input], outputs=[audio_state]).then(
        user, [audio_state, chatbot], [audio_state, chatbot]
    ).then(
        bot, chatbot, [chatbot, image_output, translation_output]
    ).then(
        lambda: gr.update(value=None), None, audio_input
    )

    clear.click(lambda: None, None, chatbot, queue=False)


ui.launch()

c:\Users\veace\anaconda3\envs\llms\Lib\site-packages\gradio\components\chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Hi, how are you today?


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\veace\anaconda3\envs\llms\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\veace\anaconda3\envs\llms\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\veace\anaconda3\envs\llms\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\veace\anaconda3\envs\llms\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\veace\anaconda3\envs\llms\Lib\site-packages\starlette\middleware\errors.py", line 187, in __call__
    raise exc
  File "c

What's the crack?


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\veace\anaconda3\envs\llms\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\veace\anaconda3\envs\llms\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\veace\anaconda3\envs\llms\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\veace\anaconda3\envs\llms\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\veace\anaconda3\envs\llms\Lib\site-packages\starlette\middleware\errors.py", line 187, in __call__
    raise exc
  File "c

I don't have any travel needs for now.


In [13]:
# Translate the following sentence to russian:

# My name is Slava and I'm a data scientist